In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

os.chdir('/home/trevor/disinformation')

In [2]:
# Import the data frames
wf = pd.read_csv('data/datasets/fake_domain_word_freq.csv')
df = pd.read_csv('data/datasets/fake_real_domains_combined.csv')
rs = pd.read_csv('data/datasets/reg_scores.csv')

In [ ]:
# Score domain keywords based on word frequencies in the sample
for e, dom in enumerate(df['dom_split']):
    dom_split = dom.split(' ')
    score = 0
    for word in dom_split:
        try:
            prob = wf.loc[wf['word'] == word, 'samp_freq'].values[0]
        except IndexError: prob = 0
        score += prob
    df.loc[e, 'dom_score'] = score
mult = 1/df['dom_score'].max()
df['dom_score'] = df['dom_score'] * mult

In [ ]:
# Score registrars based on word frequencies in the sample
for e, reg in enumerate(df['registrar']):
    score = 0
    other_val = rs.loc[rs['registrar'] == 'Other', 'reg_score'].values[0]
    try:
        prob = rs.loc[rs['registrar'] == reg, 'reg_score'].values[0]
    except IndexError: prob = other_val
    df.loc[e, 'reg_score'] = score

df.tail()

In [ ]:
# Fill NAs for rank with the highest known one
df['rank'] = df['rank'].fillna(df['rank'].max())

In [ ]:
# Create the split datasets for train, validate, and test
train, validate, test = np.split(
    df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

y = train['trust']
features = ['domain_length', 'keyword_length', 'num_nameservers',
            'dom_age_days', 'dom_score', 'reg_score', 'rank']

In [ ]:
# Check for NaN values in the sets
print(train[features].isnull().sum().sum())
print(validate[features].isnull().sum().sum())
print(test[features].isnull().sum().sum())

In [ ]:
# Train the classifier
X = pd.get_dummies(train[features])
X_test = pd.get_dummies(test[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

In [ ]:
# Create confusion matrix
pd.crosstab(test['trust'], predictions, rownames=['Actual trust'], colnames=['Predicted trust'])

In [ ]:
# View a list of the features and their importance scores
list(zip(train[features], model.feature_importances_))

In [ ]:
# Review the output
output = pd.DataFrame({'domain': test.domain,
                       'assigned_trust': test.trust,
                       'prediction': predictions,
                       'correct': test.trust == predictions})

print(output.loc[output['correct'] == False])
num_false = len(output.loc[output['correct'] == False])
samp_size = len(output)
print('Number of samples: {}'.format(samp_size))
print('Number of false negatives: {}'.format(num_false))
print('Total correct: {}%'.format(1 - (num_false/samp_size)))